# Import libraries & UDFs
## Import essentials

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import caffeine
import time
from tqdm import tqdm

import pandas as pd

import requests
from bs4 import BeautifulSoup
import json
from flatten_json import flatten

## User defined functions

### Generate tag string

In [2]:
def tag_string(tag, html_element):
    typ = html_element.split("=")[0]
    name = html_element.split("=")[1]
    return tag + f"[{typ}=" + "\"" + name + "\"]"

### Display always on

For when some cell take too long to run

In [3]:
def on(): 
    caffeine.on(display=True)

### Launch driver

In [6]:
def launch_driver():
    global browser
    
    path = {"executable_path": "/Users/marclamy/Desktop/code/chromedriver6"}

    # Starting driver. Headless displays the browser. 
    browser = Browser("chrome") 
    
    # Let's go the the results page
    browser.visit("https://www.premierleague.com/results?co=1&se=1&cl=-1")

#     # Thanks for the cookies I take them all 🍪
#     browser.find_by_tag(tag_string("div", "class=btn-primary cookies-notice-accept")).click()

#     # There is an ad that is only temporary 
#     browser.find_by_tag(tag_string("a", "id=advertClose")).click()

# Get data

To get each match, I will scrape the Premier League's website which is made as follow for each game.

*premierleague.com / match /* ***{id of the game}***

First, I need the id of each game so I can loop from the list after. 


## Get season_id 

To scrape the ids of each game, I'll first need [chrome webdriver](https://chromedriver.chromium.org/downloads).

### Start the driver

In [7]:
launch_driver()

### Accept the cookies and close the ad 

### Get each season id 

I choose from 95 as until then there were 22 teams (82 more games per year) and some seasons were incomplete, not all match were listed. 

In [8]:
all_seasons = []

for x in range(1992, 2021):
    second = f"0{x+1}" if x+1 < 10 else x+1
    all_seasons.append(f"{x}/{str(x+1)[2:]}")

In [ ]:
browser.find_by_tag(tag_string("div", "data-dropdown-current=compSeasons")).click()

In [11]:
launch_driver()

df_season_ids = pd.DataFrame(columns=["season", "season_id"])

browser.find_by_tag(tag_string("div", "data-dropdown-current=compSeasons"), wait_time=10).click()


for element in browser.find_by_tag("li"): 
    season_id = element["data-option-id"]
    season = element["data-option-name"]
    if season_id != None and season in all_seasons:
        df_season_ids.loc[len(df_season_ids)] = [season, season_id]

df_season_ids["nb_games"] = 380
df_season_ids.loc[[26, 27, 28], "nb_games"] = 462
df_season_ids

,season,season_id,nb_games
0,2020/21,363,380
1,2019/20,274,380
2,2018/19,210,380
3,2017/18,79,380
4,2016/17,54,380
5,2015/16,42,380
6,2014/15,27,380
7,2013/14,22,380
8,2012/13,21,380
9,2011/12,20,380


### Save season_id in a csv

In [14]:
df_season_ids = df_season_ids.sort_values(by="season").reset_index(drop=True)
df_season_ids.to_csv("data/season_ids.csv", index=False)

## Get match_id

In [18]:
on()

df_season_ids = pd.read_csv("data/season_ids.csv")
df_match_ids = pd.DataFrame(columns=["season", "season_id", "match_id", "nb_games"])

launch_driver()

for index, season_id in enumerate(df_season_ids["season_id"]):
    all_match_ids = set() # for unique values directly
    browser.visit(f"https://www.premierleague.com/results?co=1&se={season_id}&cl=-1")
    
    # There is a total of 380 games per season
    count = 0
    seasson_games_count = df_season_ids.loc[index, "nb_games"]
    while len(all_match_ids) != seasson_games_count: 
        tag = browser.find_by_tag("li", wait_time=10)
        all_match_ids = {e["data-comp-match-item"] for e in tag if e["data-comp-match-item"] != None} 
        
        browser.execute_script(f"window.scroll(0, document.body.scrollHeight)")
        
        if count % 2 == 0: 
            browser.execute_script(f"window.scroll(0, 0)")
            
            
        print(df_season_ids.loc[index, "season"], len(all_match_ids), count, " "*20, end="\r")
        
        count += 1
        
        if count == 40:
            all_match_ids = set()
            browser.visit(f"https://www.premierleague.com/results?co=1&se={season_id}&cl=-1")
            count = 0
        
        
    for match_id in all_match_ids:
        df_match_ids.loc[len(df_match_ids)] = [df_season_ids.loc[index, "season"], season_id, 
                                               match_id, seasson_games_count]
    
    

browser.quit()

In [19]:
print(df_match_ids.shape)
display(df_match_ids.head())

df_match_ids.to_csv("data/match_ids.csv", index=False)

(11266, 4)


,season,season_id,match_id,nb_games
0,1992/93,1,136,462
1,1992/93,1,108,462
2,1992/93,1,263,462
3,1992/93,1,101,462
4,1992/93,1,241,462


In [20]:
df_match_ids["match_id"]

0          136
1          108
2          263
3          101
4          241
         ...  
11261    59062
11262    58999
11263    59197
11264    59244
11265    58975
Name: match_id, Length: 11266, dtype: object

In [24]:
import pandas as pd
df_match_ids = pd.read_csv("data/match_ids.csv")

# df_players = pd.read_csv('data/all_players_r.csv')
# df_events = pd.read_csv('data/all_events_r.csv')
# df_games = pd.read_csv('data/all_games_r.csv')

df_players = pd.DataFrame()
df_events = pd.DataFrame()
df_games = pd.DataFrame()

# manquants = set(df_match_ids["match_id"]) - set(df_players["match_id"])


## Get matchs

In [32]:
df_players = pd.concat([df_players, playersdf], axis=0)
df_players

,matchPosition,matchShirtNumber,captain,playerId,info_position,info_positionInfo,nationalTeam_isoCode,nationalTeam_country,nationalTeam_demonym,birth_date_millis,...,name_display,name_first,name_last,id,altIds_opta,substitute,team_id,match_id,birth_place,info_shirtNum
0,G,99,False,0,G,Goalkeeper,GB-ENG,England,English,-340934400000,...,Clive Baker,Clive,Baker,199,p16899,0,8,136,NaN,NaN
1,D,99,False,0,D,Defender,GB-ENG,England,English,-157075200000,...,David Linighan,David,Linighan,206,p2729,0,8,136,NaN,NaN
2,M,99,False,0,M,Midfielder,GB-ENG,England,English,24364800000,...,Gavin Johnson,Gavin,Johnson,209,p3649,0,8,136,NaN,NaN
3,D,99,False,0,D,Defender,GB-SCT,Scotland,Scottish,-391651200000,...,John Wark,John,Wark,204,p17092,0,8,136,NaN,NaN
4,M,99,False,0,M,Midfielder,GB-ENG,England,English,-65145600000,...,Jason Dozzell,Jason,Dozzell,212,p3215,0,8,136,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,M,99,False,0,M,Midfielder,GB-ENG,England,English,89856000000,...,Garry Flitcroft,Garry,Flitcroft,300,p1249,0,11,233,NaN,NaN
19,D,99,False,0,D,Defender,IE,Ireland,Irish,-88300800000,...,Terry Phelan,Terry,Phelan,291,p1770,0,11,233,NaN,NaN
20,M,99,False,0,M,Midfielder,GB-ENG,England,English,-264038400000,...,Steve McMahon,Steve,McMahon,294,p10790,0,11,233,NaN,NaN
21,G,99,False,0,G,Goalkeeper,GB-ENG,England,English,-272073600000,...,Tony Coton,Tony,Coton,279,p15664,0,11,233,NaN,NaN


In [53]:
on()
df_match_ids = pd.read_csv("data/match_ids.csv")

# df_match_ids = pd.read_csv("real_data/match_ids.csv")

# df_players = pd.read_csv('real_data/all_players_92.csv')
# df_events = pd.read_csv('real_data/all_events_92.csv')
# df_games = pd.read_csv('real_data/all_games_92.csv')

df_players = pd.DataFrame()
df_events = pd.DataFrame()
df_games = pd.DataFrame()

# old = df_match_ids["match_id"]
# new = df_players["match_id"]

# ids = set(old) - set(new)


ids = df_match_ids["match_id"]
# manquants = set(df_match_ids["match_id"]) - set(df_games["match_id"])


print(len(ids))
# print(fghjds)



# the code is running only the first 400 ids right now, remove that if you dont wanna test it
for index, match_id in enumerate(tqdm(ids)):
    url = 'https://www.premierleague.com/match/' + str(match_id)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')
    data = json.loads(soup.find('div', class_='mcTabsContainer').attrs['data-fixture'])

    players = []
    teams = []
    for team in data['teamLists']:
        team_id = team["teamId"]
    
        for player in team["lineup"]:
            player = flatten(player)
            player["substitute"] = "0"
            player["team_id"] = team_id
            players.append(flatten(player))

        for player in team["substitutes"]:
            player = flatten(player)
            player["substitute"] = "1"
            player["team_id"] = team_id
            players.append(flatten(player))
            
    playersdf = pd.DataFrame(players)
    playersdf['match_id'] = match_id
    df_players = pd.concat([df_players, playersdf], axis=0)

    events = []
    for event in data['events']:
        events.append(flatten(event))
    eventsdf = pd.DataFrame(events)
    eventsdf['match_id'] = match_id
    df_events = pd.concat([df_events, eventsdf], axis=0)

    data['teamLists'] = ''
    data['events'] = ''
    match = pd.DataFrame([flatten(data)])
    match['match_id'] = match_id
    df_games = pd.concat([df_games, match], axis=0)
    
    
#     if index % 50 == 0:
    df_games.to_csv('data/all_games.csv', index=False)
    df_players.to_csv('data/all_players.csv', index=False)
    df_events.to_csv('data/all_events.csv', index=False)